# モジュール1: RAG（検索拡張生成）

このモジュールでは、Amazon Bedrock EmbeddingモデルとTiDB Cloud Starterベクトル検索を使用してデータを取得します。その後、LLM（大規模言語モデル）を使って質問への回答を生成します。

ここでは、`pytidb`とTiDB Cloud Starterを使って、どれだけ簡単にRAGアプリケーションを構築できるかを示します。

> **注意:**
>
> - 環境パラメータに `SERVERLESS_CLUSTER_HOST`、`SERVERLESS_CLUSTER_PORT`、`SERVERLESS_CLUSTER_USERNAME`、`SERVERLESS_CLUSTER_PASSWORD`、`SERVERLESS_CLUSTER_DATABASE_NAME` をすでに設定しています。
> - また、このラボでAmazon Bedrockの利用権限も付与済みです。TiDB Labsプラットフォーム以外でこのコードスニペットを使用したい場合は、事前にこれらを設定してください。

## 依存関係のインストール

In [ ]:
%pip install -q \
    pytidb==0.0.10.dev1 \
    boto3==1.38.23 \
    litellm \
    pandas

## データベースクライアントの初期化

In [ ]:
import os

from litellm import completion
from typing import Optional, Any
from pytidb import TiDBClient
from pytidb.schema import TableModel, Field
from pytidb.embeddings import EmbeddingFunction

db = TiDBClient.connect(
    host=os.getenv("SERVERLESS_CLUSTER_HOST"),
    port=int(os.getenv("SERVERLESS_CLUSTER_PORT")),
    username=os.getenv("SERVERLESS_CLUSTER_USERNAME"),
    password=os.getenv("SERVERLESS_CLUSTER_PASSWORD"),
    database=os.getenv("SERVERLESS_CLUSTER_DATABASE_NAME"),
    enable_ssl=True,
)

embedding_model = "bedrock/amazon.titan-embed-text-v2:0"

text_embedding_function = EmbeddingFunction(
    embedding_model,
    timeout=60
)

## コンテキストの準備

この場合、コンテキストはドキュメントです。openaiの埋め込みモデルを使ってドキュメントの埋め込みを取得し、それらをTiDBに保存します。

In [ ]:
table_name = "documents"
class Document(TableModel, table=True):
    __tablename__ = table_name
    __table_args__ = {"extend_existing": True}
    id: int | None = Field(default=None, primary_key=True)
    text: str = Field(max_length=1024)
    embedding: Optional[Any] = text_embedding_function.VectorField(
        source_field="text",
    )

documents = [
    Document(id=1, text="TiDB is an open-source distributed SQL database that supports Hybrid Transactional and Analytical Processing (HTAP) workloads."),
    Document(id=2, text="TiFlash is the key component that makes TiDB essentially an Hybrid Transactional/Analytical Processing (HTAP) database. As a columnar storage extension of TiKV, TiFlash provides both good isolation level and strong consistency guarantee."),
    Document(id=3, text="TiKV is a distributed and transactional key-value database, which provides transactional APIs with ACID compliance. With the implementation of the Raft consensus algorithm and consensus state stored in RocksDB, TiKV guarantees data consistency between multiple replicas and high availability. "),
]

table = db.create_table(schema=Document, if_exists="overwrite")
table.bulk_insert(documents)

## ベクトルのコサイン距離による検索

質問とドキュメントの埋め込みを比較して、TiDBから関連するドキュメントを取得します。

In [ ]:
question = "what is TiKV?"

results = table.search(question).limit(1)
results.to_pandas()

## 回答の生成

In [ ]:
from litellm import completion

llm_model = "bedrock/us.amazon.nova-lite-v1:0"

messages = [
    {"role": "system", "content": f"Please carefully answer the question by {str(results)}"},
    {"role": "user", "content": question}
]

llm_response = completion(
    model=llm_model,
    messages=messages,
)

print(llm_response.choices[0].message.content)